<a href="https://colab.research.google.com/github/Shrinjita/AI-ML-Portfolio/blob/main/BAJAJ_FINSERV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Step 1: Import required libraries
import json
import pandas as pd

# Step 2: Load the JSON file
file_path = '/content/drive/MyDrive/DataEngineeringQ2.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# Step 3: Quick inspection
print(f"Total records: {len(data)}")
print(json.dumps(data[0], indent=2))  # Display the first entry nicely


Total records: 31
{
  "_id": "T6hf3rb5",
  "appointmentId": "40d2-9c9f",
  "patientDetails": {
    "_id": "T6hb630b3",
    "firstName": "Css",
    "lastName": "",
    "emailId": "",
    "gender": "",
    "alternateContact": "",
    "birthDate": null
  },
  "phoneNumber": "96686896670",
  "consultationData": {
    "adviceTemplates": [],
    "advices": [],
    "attachments": [],
    "chiefComplaints": [],
    "customOne": [],
    "customThree": [],
    "customTwo": [],
    "disease": [],
    "doctorNotes": "",
    "emergencyInstructions": [],
    "emergencyInstructionsTemplate": [],
    "emrTemplates": [],
    "examinationNote": [],
    "findings": [],
    "investigationInstructions": [],
    "investigationTemplates": [],
    "investigations": [],
    "isBalicAppointment": false,
    "isQuickPrescription": false,
    "labTest": [],
    "languageCode": "en",
    "medicineTemplates": [],
    "medicines": [
      {
        "medicineId": "619404",
        "medicineName": "A",
        "medici

In [11]:
# Extract patient details + phone + consultation (basic flattening)
records = []
for entry in data:
    record = {
        'appointmentId': entry['appointmentId'],
        'patientName': entry['patientDetails'].get('firstName', '') + " " + entry['patientDetails'].get('lastName', ''),
        'phoneNumber': entry['phoneNumber'],
        'gender': entry['patientDetails'].get('gender', ''),
        'birthDate': entry['patientDetails'].get('birthDate', ''),
        'medicines': [med['medicineName'] for med in entry['consultationData'].get('medicines', [])]
    }
    records.append(record)

df = pd.DataFrame(records)
df.head()


,appointmentId,patientName,phoneNumber,gender,birthDate,medicines
0,40d2-9c9f,Css,96686896670,,None,"[A, B]"
1,40dbtc9f,Lokesh,9496368916,M,1996-05-16T18:30:00.000Z,"[A, C]"
2,g3Wt5c9f,Shila Das,7787204833,,,"[B, A, C]"
3,fb6-a535,Bhavika Ben Panchal,9376756879,F,1988-04-24T14:30:00.000Z,[B]
4,67h7KL9,Raghu Viju,5267384241,,,"[D, A]"


In [12]:
import json
import pandas as pd
from datetime import datetime
from collections import Counter
df = pd.json_normalize(data)

# Fill gender using mode
mode_gender = df['patientDetails.gender'].mode()[0]
df['patientDetails.gender'] = df['patientDetails.gender'].replace('', mode_gender).fillna(mode_gender)
female_percentage = round((df['patientDetails.gender'] == 'F').mean() * 100, 2)

# Calculate age
today = datetime(2025, 4, 24)
def calculate_age(birthdate):
    if pd.isna(birthdate) or birthdate == '':
        return None
    return (today - pd.to_datetime(birthdate).tz_localize(None)).days // 365

df['age'] = df['patientDetails.birthDate'].apply(calculate_age)

# Categorize into age groups
def age_group(age):
    if age is None:
        return 'Unknown'
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    elif age <= 59:
        return 'Adult'
    else:
        return 'Senior'

df['ageGroup'] = df['age'].apply(age_group)
adult_count = (df['ageGroup'] == 'Adult').sum()
avg_age = int(df['age'].dropna().mean())

# Average number of medicines
df['medicineCount'] = df['consultationData.medicines'].apply(lambda x: len(x))
avg_medicines = round(df['medicineCount'].mean(), 2)

# 3rd most frequently prescribed medicine
all_meds = [med['medicineName'] for sublist in df['consultationData.medicines'] for med in sublist]
med_counts = Counter(all_meds)
third_most_common = med_counts.most_common(3)[2][0]

# Total active meds for "AFTER MEAL"
def has_after_meal(meds):
    return any(m.get('instruction') == 'AFTER MEAL' for m in meds)
def count_active_meds(meds):
    return sum(1 for m in meds if m.get('isActive'))

df['hasAfterMeal'] = df['consultationData.medicines'].apply(has_after_meal)
df['activeCount'] = df['consultationData.medicines'].apply(count_active_meds)
total_active_after_meal = df[df['hasAfterMeal']]['activeCount'].sum()

# Patients with only A (90 DAYS, active)
def match_criteria(row):
    meds = row['consultationData.medicines']
    has_A_90 = any(m.get('medicineName') == 'A' and m.get('duration') == '90' and m.get('durationIn') == 'DAYS' and m.get('isActive') for m in meds)
    has_other_active = any(m.get('isActive') and m.get('medicineName') != 'A' for m in meds)
    return has_A_90 and not has_other_active

criteria_patients = df[df.apply(match_criteria, axis=1)]

# Most common active medicine
def active_meds(row):
    return {m['medicineName'] for m in row['consultationData.medicines'] if m.get('isActive')}
df['activeMeds'] = df.apply(active_meds, axis=1)
flat_active = [med for meds in df['activeMeds'] for med in meds]
most_active_medicine = Counter(flat_active).most_common(1)[0][0]

# Display all answers
print("Q4 Female %:", female_percentage)
print("Q5 Adult Count:", adult_count)
print("Q6 Avg Medicines:", avg_medicines)
print("Q7 3rd Most Frequent Medicine:", third_most_common)
print("Q8 Average Age:", avg_age)
print("Q9 Total Active Meds w/ AFTER MEAL:", total_active_after_meal)
print("Q10 Matching Patients:", len(criteria_patients))
print("Q11 Most Prescribed Active Medicine:", most_active_medicine)


Q4 Female %: 32.26
Q5 Adult Count: 21
Q6 Avg Medicines: 2.13
Q7 3rd Most Frequent Medicine: C
Q8 Average Age: 34
Q9 Total Active Meds w/ AFTER MEAL: 46
Q10 Matching Patients: 3
Q11 Most Prescribed Active Medicine: A


In [13]:
def is_valid_indian_number(number):
    if not isinstance(number, str):
        return False

    # Remove prefix if present
    if number.startswith('+91'):
        number = number[3:]
    elif number.startswith('91'):
        number = number[2:]

    # Check if the remaining is exactly 10 digits and is numeric
    if len(number) != 10 or not number.isdigit():
        return False

    # Check if the number is in the valid range
    num = int(number)
    return 6000000000 <= num <= 9999999999

# Apply validation
df['isValidMobile'] = df['phoneNumber'].apply(is_valid_indian_number)

# Count valid numbers
valid_count = df['isValidMobile'].sum()
print("Total valid phone numbers:", valid_count)


Total valid phone numbers: 18


In [14]:
# Drop rows with missing values in either column
corr_df = df[['medicineCount', 'age']].dropna()

# Calculate Pearson correlation
pearson_corr = corr_df['medicineCount'].corr(corr_df['age'])
# Round to 2 decimal places
pearson_corr = round(pearson_corr, 2)
print("Pearson correlation:", pearson_corr)


Pearson correlation: -0.21
